In [ ]:
import pandas as pd
import matplotlib

In [ ]:
%run data.ipynb
%run ../modules/preprocessing.ipynb
%run ../modules/cds.ipynb
%run ../modules/utils.ipynb

In [ ]:
# df15 = load_uc2(15)
df15 = pd.read_excel('/Users/bryan/Documents/ChenLab/10Jun2021/degraded/DrChen_tRNA_A_Lys_FA_15.xlsx')
df15 = thermo_df(df15)
df16 = load_uc2(16)
df21 = load_uc2(21)
df22 = load_uc2(22)
df15.shape, df16.shape, df21.shape, df22.shape

In [ ]:
df15.info()

In [ ]:

df_src = df16.copy()
# df_src = df_chosen.copy()
plotly_zone(df_src)

In [ ]:
# direct sequencing

df_top = df_src[(df_src.Mass>6000)&(df_src.Mass<8800)].sort_values('Vol', ascending=False).iloc[:100].copy()
# df_top = df_src.copy()
# plotly_zone(df_top4
df_salts = list()
for salt in [21.9819, 37.9558, 34.9694, 27.9949, 56.92]:
    df_salt = match_dfs(df_top, df_top, shift=salt)
#     df_top = df_top.drop(df_salt.index)
    df_salts.append(df_salt)
df_salts = pd.concat(df_salts)
# plotly_zones(df_top, df_salts)

bcr = base_calling_random(df_top)
plotly_basecalling(*bcr, annotate=False)
# groups = basecalling_groups(df_top)
# plotly_basecalling(*groups[0], annotate=False)

dfm1 = match_dfs(bcr[0], df_src, shift=-1*M)
dfm2 = match_dfs(bcr[0], df_src, shift=1*M)
# plotly_multi_zones([bcr[0], dfm1, dfm2])
plotly_zones(bcr[0], dfm1)

In [ ]:
target = 3982
dfa = df_src[(df_src.Mass>target-1)&(df_src.Mass<target+1)]
target += M
dfb = df_src[(df_src.Mass>target-1)&(df_src.Mass<target+1)]
dfa, dfb

In [ ]:
# Choose dots

idxs = list()
# create our callback function
def on_selection(trace, points, selector):
    print('points {}'.format(points.point_inds) )
    idxs.extend(points.point_inds)

def on_click(trace, points, selector):
#     print('points {}'.format(points.point_inds) )
    idxs.extend(points.point_inds)

df_sample = df_top #df_src[(df_src.Mass>5000)&(df_src.Mass<8700)]
f = zone_selection(df_sample, on_selection=on_selection, on_click=on_click)
f

In [ ]:
df_chosen = df_sample.iloc[idxs]
# df_zone = df_sample.drop(df_chosen.index)
df_chosen.shape

In [ ]:
df5p = df_chosen.copy()

In [ ]:
df_sample = df5p.sort_values('Vol', ascending=False).iloc[:20]
# df_sample = df5p
orientation = 5

# df_sf, df_salts = remove_salts(df_sample)
df_sf = df_sample

bcr = base_calling_random(df_sf)
plotly_basecalling(*bcr, annotate=False, y='RT')
group = bcr

# groups = basecalling_groups(df_sf)
# group = groups[0]
# plotly_basecalling(*group, annotate=False)

# output seats
# df = process_mass_base_seats(*group, full_mass=24174, orientation=orientation)
# df.to_excel('~/Downloads/seats.xlsx')

dfm = match_dfs(bcr[0], df_src, shift=-1*M)
if not dfm.empty:
    plotly_zones(bcr[0], dfm, y='RT')

In [ ]:
target = 8650
dfa = df_src[(df_src.Mass>target-1)&(df_src.Mass<target+1)]
target += M
dfb = df_src[(df_src.Mass>target-1)&(df_src.Mass<target+1)]
dfa, dfb

In [ ]:
# bcr[0].to_excel('~/Downloads/type_position_ori.xlsx')
dfa = bcr[0].copy()
dfb = dfm.copy()
dfa['DeMethyl'] = False
dfb['DeMethyl'] = True
dfab = pd.concat([dfa, dfb])
dfab.to_excel('~/Downloads/type_position_ori.xlsx')

In [ ]:
def remove_salts(df_sample, salts=[21.9819, 37.9558, 34.9694, 27.9949, 56.92]):
    df = df_sample.copy()
    df_salts = list()
    for salt in salts:
        df_salt = match_dfs(df, df, shift=salt)
#         df = df.drop(df_salt.index)
        df_salts.append(df_salt)
    df_salts = pd.concat(df_salts).drop_duplicates()
    df = df.drop(df_salts.index)
    return df, df_salts